In [2]:
!pip install rake_nltk
!pip install surprise

  Running setup.py bdist_wheel for rake-nltk ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 3.3MB 14.1MB/s ta 0:00:01
    100% |████████████████████████████████| 286kB 41.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for scikit-surprise ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import datetime
import gzip
import pandas as pd
from rake_nltk import Rake
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise import *
from surprise import accuracy
from surprise.model_selection import *

In [4]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')


k_core_dir = './data/5-core/'
metadata_dir = './data/metadata/'

In [5]:
reviews_home = getDF(k_core_dir + 'reviews_Home_and_Kitchen_5.json.gz')
reviews_home['reviewTime'] = reviews_home.unixReviewTime.apply(lambda d: datetime.datetime
                                                       .fromtimestamp(int(d))
                                                       .strftime('%Y-%m-%d'))
reviews_home = reviews_home.sort_values(['reviewTime', 'reviewerID', 'asin']).reset_index(drop=True)
reviews_home.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A12XRHXJRFNKPT,B00004RFKS,"M. M. Davis ""Matt Davis""","[15, 16]","This knife is made of lots of good, hard steel...",5.0,A truly excellent knife,957225600,2000-05-02
1,A1CKLHQQG32AQ9,B00004SPED,"J. Hauer ""me""","[55, 71]",This can opener was given to us as a wedding p...,1.0,Not really all that useful,957225600,2000-05-02
2,A1LUUO72VAFKFJ,B00004SGE2,T. K. Schonhoff,"[264, 271]",When we went out to buy our food processor I w...,5.0,Blows away the Cuisinart,957225600,2000-05-02
3,A25PD5YQS1E5Z9,B00004S9EQ,Helena,"[48, 52]",I love this chopper. I use it to chop all kind...,4.0,Great chopper!,957225600,2000-05-02
4,A3H78NCT3DJMY2,B00004RFKS,Barnaby Dorfman,"[10, 10]",This knife will replace half the gadgets and a...,5.0,All you need,957225600,2000-05-02


In [6]:
product_home = getDF(metadata_dir + 'meta_Home_and_Kitchen.json.gz')
product_home['categories'] = product_home['categories'].apply(lambda x: x[0])
product_home.head()

,asin,salesRank,imUrl,categories,title,description,related,price,brand
0,0076144011,{'Books': 6285595},http://g-ecx.images-amazon.com/images/G/01/x-s...,[Home & Kitchen],"Ninjas, Piranhas, and Galileo",NaN,NaN,NaN,NaN
1,0130350591,{'Kitchen & Dining': 459680},http://ecx.images-amazon.com/images/I/21zcx6RC...,"[Home & Kitchen, Kitchen & Dining, Dining & En...",Le Creuset Kiwi (Green) Butter Dish Stoneware,Each piece of Le Creuset dinnerware is crafted...,NaN,NaN,NaN
2,0307394530,"{'Arts, Crafts & Sewing': 3597}",http://ecx.images-amazon.com/images/I/51A4FWuj...,"[Home & Kitchen, Artwork, Posters & Prints]",Martha Stewart's Wedding Cakes,Of all the decisions that go into planning a w...,"{'also_bought': ['144630163X', '1402717733', '...",14.99,Random House
3,0439903491,{'Software': 7065},http://ecx.images-amazon.com/images/I/61tVxcko...,"[Home & Kitchen, Artwork, Posters & Prints]",NaN,Shiver me timbers! Solve I SPY pirate picture ...,"{'also_viewed': ['B000YFSZU8', 'B0016KWZB0', '...",29.99,NaN
4,0578060604,NaN,http://ecx.images-amazon.com/images/I/41aCELWJ...,"[Home & Kitchen, Furniture, Kitchen & Dining R...",Build A Maloof Inspired Low Back Dining Chair ...,The Maloofinspired Low Back Dining Chair is no...,{'also_viewed': ['B004IO6RS8']},NaN,NaN


In [7]:
len(reviews_home), reviews_home.reviewerID.nunique(), reviews_home.asin.nunique()

(551682, 66519, 28237)

In [28]:
# particular user
user = 'A12XRHXJRFNKPT'
item = product_home[product_home['asin'] == 'B002Q5YH9C'].title.values[0]

## Simple recommendation system

In [9]:
metadata = reviews_home.groupby(['asin']) \
                       .agg({'asin':'size', 'overall':'mean'}) \
                       .rename(columns={'asin':'vote_count','overall':'vote_average'}) \
                       .reset_index()
metadata = pd.merge(metadata, product_home, how='left', on='asin')
s = metadata.apply(lambda x: pd.Series(x['categories']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'category'
gen_metadata = metadata.drop('categories', axis=1).join(s)
gen_metadata.head(3).transpose()

,0,0,0
asin,0615391206,0615391206,0615391206
vote_count,11,11,11
vote_average,4.45455,4.45455,4.45455
salesRank,{'Kitchen & Dining': 3900},{'Kitchen & Dining': 3900},{'Kitchen & Dining': 3900}
imUrl,http://ecx.images-amazon.com/images/I/41EEhiqv...,http://ecx.images-amazon.com/images/I/41EEhiqv...,http://ecx.images-amazon.com/images/I/41EEhiqv...
title,Zoku Quick Pops Recipe Book,Zoku Quick Pops Recipe Book,Zoku Quick Pops Recipe Book
description,This book will help you master the art of crea...,This book will help you master the art of crea...,This book will help you master the art of crea...
related,"{'also_bought': ['B004G7W8BW', 'B004KS5BQ6', '...","{'also_bought': ['B004G7W8BW', 'B004KS5BQ6', '...","{'also_bought': ['B004G7W8BW', 'B004KS5BQ6', '..."
price,17.29,17.29,17.29
brand,Zoku,Zoku,Zoku


In [35]:
def build_chart(category, percentile=0.90):
    df = gen_metadata[gen_metadata['category'] == category]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & 
                   (df['vote_average'].notnull())][['asin', 'title', 'vote_count', 'vote_average']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: 
                        (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C),
                        axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified


build_chart('Home & Kitchen').head(15)

,asin,title,vote_count,vote_average,wr
28159,B00K4S1L1K,Nonstick Silicone Baking Mat eliminates burnt ...,48,5,4.448033
28115,B00JOUZOTY,Dough Blender - Pastry Blender - Stainless Ste...,42,5,4.408132
1467,B00006JSUA,"Lodge L8SK3 Pre-Seasoned Cast-Iron Skillet, 10...",1064,4,3.992647
12999,B001N07KUE,"EatSmart Precision Pro Digital Kitchen Scale, ...",921,4,3.991554
1417,B00006IUWA,"Presto 04820 PopLite Hot Air Popper, White",662,4,3.988442
871,B00005B8K5,"Pyrex Storage 10-Piece Set, Clear with Blue Lids",656,4,3.988343
8646,B000QFDNP8,Contigo AUTOSEAL Stainless Steel Vacuum Insula...,634,4,3.987963
4182,B0006HUYGM,"Eureka EasyClean Corded Hand-Held Vacuum, 71B",612,4,3.987557
1794,B00009R66F,Hoover SteamVac Carpet Cleaner with Clean Surg...,608,4,3.987481
520,B00004SPEU,KRUPS F20342 Electric Spice and Coffee Grinder...,592,4,3.987164


## Content based recommender system

In [33]:
metadata['description'] = metadata['description'].fillna('')

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(metadata['description'])

tfidf_matrix.shape

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# metadata = metadata.reset_index()
titles = metadata['title']
indices = pd.Series(metadata.index, index=metadata['title'])


def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]


get_recommendations(item).head(10)

1      Good Night, Sweet Butterflies: A Color Dreamland
2                          Wilton Decorating Cakes Book
3     Vandor 52423 Star Wars Large Recycled Shopper ...
4                           Franklin SPELLING CORRECTOR
5                 Mighty Bright Fold-n-Stow Book Holder
6     Best Choice 2013 Design Fist Cup Brass Knuckle...
7     World Pride Mickey Mouse Face Shape Cookie Cutter
8     Tupperware Fresh &amp; Pure Ice Tray Purple Se...
9              Tupperware Orange Citrus Peeler Set of 6
10         Tupperware Smidgets in Pink and Red Set of 5
Name: title, dtype: object

In [34]:
metadata['keywords'] = ""

for index, row in metadata.iterrows():
    description = row['description']
    r = Rake()
    r.extract_keywords_from_text(description)
    key_words_dict_scores = r.get_word_degrees()
    metadata.at[index, 'keywords'] = list(key_words_dict_scores.keys())


def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


features = ['keywords', 'categories', 'brand']

for feature in features:
    metadata[feature] = metadata[feature].fillna('').apply(clean_data)

metadata['soup'] = metadata['keywords'] + metadata['categories'] + metadata['brand'].apply(lambda x: [x])
metadata['soup'] = metadata['soup'].apply(lambda x: ' '.join(x))

count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)

# metadata = metadata.reset_index()
titles = metadata['title']
indices = pd.Series(metadata.index, index=metadata['title'])

get_recommendations(item).head(10)

15403              Curious Chef Medium Nylon Plastic Knife
3207        Wusthof Le Cordon Bleu 6 inch Cooks Chef Knife
2083     Victorinox 47521 10-Inch Chef's Knife, Black F...
2098     Victorinox 40570 Cutlery 6-Inch Chef's Knife, ...
6064     Kyocera Revolution Series  6-1/4 inch, 16 cm, ...
7587            MAC MTH-80 8&quot; Chef's Knife w/ Dimples
8378     Kyocera Revolution Series 7-Inch Professional ...
8630     Victorinox 5-Inch Mini-Chef's Knife with Fibro...
10868    Victorinox Cutlery 3-Piece Chef's Set, Black F...
11338    Victorinox Cutlery 7-1/2-Inch Wavy Edge Chef's...
Name: title, dtype: object

In [36]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    product_indices = [i[0] for i in sim_scores]
    
    product = metadata.iloc[product_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = product[product['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = product[product['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = product[(product['vote_count'] >= m) & (product['vote_count'].notnull()) & (product['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(lambda x: 
                        (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C),
                        axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified


improved_recommendations(item).head(10)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

,title,vote_count,vote_average,wr
6064,"Kyocera Revolution Series 6-1/4 inch, 16 cm, ...",18,4,4.038621
3207,Wusthof Le Cordon Bleu 6 inch Cooks Chef Knife,20,4,4.036522
24769,"Chicago Cutlery DesignPro Chef Knives, 8-Inch",20,4,4.036522
10868,"Victorinox Cutlery 3-Piece Chef's Set, Black F...",22,4,4.034639
8378,Kyocera Revolution Series 7-Inch Professional ...,35,4,4.025946
3939,John Boos Chop-N-Slice 20-by-15-Inch Maple Cut...,37,4,4.024981
8630,Victorinox 5-Inch Mini-Chef's Knife with Fibro...,41,4,4.023253
2098,"Victorinox 40570 Cutlery 6-Inch Chef's Knife, ...",59,4,4.017731
2083,"Victorinox 47521 10-Inch Chef's Knife, Black F...",61,4,4.017275
25330,Paula Deen Signature Cutlery 3-Piece Chef Set ...,21,3,3.480000


## Collaborative Filtering

In [22]:
# Load the full dataset.
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(reviews_home[['reviewerID', 'asin', 'overall']], reader)  # change kcore_all to any of the individual kcore
raw_ratings = data.raw_ratings

# A = 80% of the data, B = 20% of the data
threshold = int(.8 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# retrain on the whole set A
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on trainset,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on testset,', end=' ')
accuracy.rmse(predictions)

Biased accuracy on trainset,   RMSE: 0.7380
Unbiased accuracy on testset, RMSE: 1.0109


1.0109210442520726

In [23]:
reader = Reader()
data = Dataset.load_from_df(reviews_home[['reviewerID', 'asin', 'overall']], reader)

algo = SVD()
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)

trainset = data.build_full_trainset()
algo.fit(trainset)

In [24]:
reviews_home[reviews_home['reviewerID'] == 'A12XRHXJRFNKPT']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A12XRHXJRFNKPT,B00004RFKS,"M. M. Davis ""Matt Davis""","[15, 16]","This knife is made of lots of good, hard steel...",5.0,A truly excellent knife,957225600,2000-05-02
14987,A12XRHXJRFNKPT,B000K8WOLK,"M. M. Davis ""Matt Davis""","[4, 4]",My wife and I both dislike the curlicues that ...,4.0,Big and simple,1176249600,2007-04-11
80882,A12XRHXJRFNKPT,B000GWE81Y,"M. M. Davis ""Matt Davis""","[0, 0]",This dining room set is solid and attractive. ...,4.0,"Good value, with lots of assembly",1296345600,2011-01-30
86119,A12XRHXJRFNKPT,B002Q5YH9C,"M. M. Davis ""Matt Davis""","[0, 0]","Since I got these, my two kids have been using...",5.0,Lets the kids get involved in the kitchen,1300406400,2011-03-18
319240,A12XRHXJRFNKPT,B00004OCO6,"M. M. Davis ""Matt Davis""","[1, 2]",This set is well designed. Each cup hooks onto...,4.0,Only one tiny complaint,1375488000,2013-08-03


In [25]:
all_items = reviews_home.asin.unique()
already_bought = reviews_home[reviews_home['reviewerID'] == 'A12XRHXJRFNKPT']['asin'].values
not_bought = [x for x in all_items if x not in already_bought]

testset = pd.DataFrame(not_bought, columns=['asin'])
testset['reviewerID'] = 'A12XRHXJRFNKPT'

for idx in range(len(testset)):
  testset.at[idx, 'pred'] = algo.predict(testset.at[idx, 'reviewerID'], testset.at[idx, 'asin']).est

testset = testset.sort_values('pred', ascending=False)

pd.merge(testset[:10], product_home[['asin', 'title']], how='inner', on='asin')[['title', 'asin', 'pred']]

,title,asin,pred
0,Microplane 38004 Professional Fine Spice Grater,B00009WE3Z,5.0
1,"Finum Goldton Filters, Blue and Green",B001D07MCU,5.0
2,"USA Pans 9 x 4 x 4 Inch Pullman, Aluminized St...",B001TO3CN8,5.0
3,Convenience Concepts 157002 Go-Accsense 3-Shel...,B000W9TVEK,5.0
4,Norpro 18 Inch Aluminum Magnetic Knife Bar,B0014WNVHC,5.0
5,Norpro 9-Inch Stainless Steel pie Pan,B000E3DTHE,5.0
6,"USA Pans 13 x 4 x 4 Inch Pullman, Aluminized S...",B002UNMZPI,5.0
7,ThermoWorks ThermoPop Super-Fast Thermometer w...,B00HZVJM3S,5.0
8,Mercer Cutlery Genesis 6-Piece Forged Knife Bl...,B000IBU9FW,5.0
9,Aeropress Micro-filters 2 Pack 700ct with Liqu...,B008MAZZPA,5.0


## Hybrid recommender system

In [37]:
id_map = metadata.set_index('title')
indices_map = id_map.set_index('asin')

In [38]:
def hybrid(user_id, title):
    idx = indices[title]
    product_id = id_map.loc[title]['asin']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    product_indices = [i[0] for i in sim_scores]
    products = metadata.iloc[product_indices][['title', 'asin', 'vote_count', 'vote_average']]
    products['est'] = products['asin'].apply(lambda x: algo.predict(user_id, x).est)
    products = products.sort_values('est', ascending=False)
    return products.head(10)


hybrid(user, item)

,title,asin,vote_count,vote_average,est
9518,"John Boos Mystery Oil 16-Ounce Bottles, 3-Case",B000VFOFXC,16,4.875000,4.933838
7587,MAC MTH-80 8&quot; Chef's Knife w/ Dimples,B000KXBSRQ,15,4.800000,4.718885
11338,Victorinox Cutlery 7-1/2-Inch Wavy Edge Chef's...,B0019WURPG,6,4.833333,4.717528
2399,John Boos 12-Inch Square Maple Cutting Board w...,B0000DDXCV,10,4.900000,4.667310
6064,"Kyocera Revolution Series 6-1/4 inch, 16 cm, ...",B000F7UZEE,18,4.722222,4.649293
2098,"Victorinox 40570 Cutlery 6-Inch Chef's Knife, ...",B0000CFDD5,59,4.745763,4.623741
3207,Wusthof Le Cordon Bleu 6 inch Cooks Chef Knife,B0001WN8K6,20,4.800000,4.593879
2083,"Victorinox 47521 10-Inch Chef's Knife, Black F...",B0000CF8YO,61,4.852459,4.589676
8630,Victorinox 5-Inch Mini-Chef's Knife with Fibro...,B000QCLEFC,41,4.634146,4.583917
24769,"Chicago Cutlery DesignPro Chef Knives, 8-Inch",B008JCGK6Y,20,4.550000,4.580275


In [ ]:
# reviews_home = getDF(k_core_dir + 'reviews_Home_and_Kitchen_5.json.gz')
# reviews_grocery = getDF(k_core_dir + 'reviews_Grocery_and_Gourmet_Food_5.json.gz')
# reviews_tools = getDF(k_core_dir + 'reviews_Tools_and_Home_Improvement_5.json.gz')

# reviews_home = pd.concat([reviews_home, reviews_grocery, reviews_tools])
# reviews_home['reviewTime'] = reviews_home.unixReviewTime.apply(lambda d: datetime.datetime
#                                                        .fromtimestamp(int(d))
#                                                        .strftime('%Y-%m-%d'))
# reviews_home = reviews_home.sort_values(['reviewTime', 'reviewerID', 'asin']).reset_index(drop=True)


# product_home = getDF(metadata_dir + 'meta_Home_and_Kitchen.json.gz')
# product_home['categories'] = product_home['categories'].apply(lambda x: x[0])

# product_grocery = getDF(metadata_dir + 'meta_Grocery_and_Gourmet_Food.json.gz')
# product_grocery['categories'] = product_grocery['categories'].apply(lambda x: x[0])

# product_tools = getDF(metadata_dir + 'meta_Tools_and_Home_Improvement.json.gz')
# product_tools['categories'] = product_tools['categories'].apply(lambda x: x[0])

# product_home = pd.concat([product_home, product_grocery, product_tools])


# metadata = reviews_home.groupby(['asin']) \
#                        .agg({'asin':'size', 'overall':'mean'}) \
#                        .rename(columns={'asin':'vote_count','overall':'vote_average'}) \
#                        .reset_index()

# m = 20

# metadata = metadata[metadata['vote_count'] >= m]
# reviews_home = reviews_home[reviews_home['asin'].isin(metadata['asin'])]

## Useful references

https://github.com/jalajthanaki/Movie_recommendation_engine/blob/master/Movie_recommendation_engine.ipynb